In [1]:
# 1. 리트리브를 위한 리트리버 : 크로마 디비 접근을 할 수 있게 가져온다.

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


# KURE-v1 임베딩 설정
model_name = "nlpai-lab/KURE-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# ✅ 기존 벡터 스토어를 불러오기
vector_store = Chroma(
    embedding_function=embeddings_function,  # ✅ HuggingFaceEmbeddings 사용
    collection_name="income_tax_collection",
    persist_directory="./income_tax_collection"  # 저장된 벡터 스토어 경로
)

# ✅ Retrieval 객체 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [2]:
# 2. 에이전트 스테이트도 가져온다.

from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph # graph_builder까지 선언해 주도록 하겠다.

class AgentState(TypedDict):
    query: str # 질문
    context: List[Document] # 컨텍스트(답변할 때 참고할 문서들: langchain의 Document 타입) - 경로는 3.1의 from langchain_core.documents import Document
    answer: str # 답변


# graph_builder 선언.
graph_builder = StateGraph(AgentState)

In [3]:
# 3. 리트리브도 가져온다.

def retrieve(state: AgentState) -> AgentState:
    query = state['query'] # 사용자의 질문을 받아온 다음
    docs = retriever.invoke(query) # 기반으로 리트리버에 대해서 검색을 하고
    return {'context': docs} # state의 컨텍스트에 넣어준다.


In [4]:
# 4. 여러 프롬프트에 활용되는 LLM도 가져온다.

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')


In [5]:
# 5. RAG 프롬프트 활용해서 답변을 생성하는 generate노드

from langchain import hub

# RAG 프롬프트를 가져옵니다.
generate_prompt = hub.pull("rlm/rag-prompt")
generate_llm = ChatOpenAI(model='gpt-4o-mini', max_completion_tokens=100)

def generate(state: AgentState):
    context = state['context']  # state에서 문맥을 추출합니다.
    query = state['query']      # state에서 사용자의 질문을 추출합니다.
    
    # RAG 체인을 구성합니다.
    rag_chain = generate_prompt | generate_llm
    
    # 질문과 문맥을 사용하여 응답을 생성합니다.
    response = rag_chain.invoke({'question': query, 'context': context})
    
    return {'answer': response.content}  # 생성된 응답을 포함하는 state를 반환합니다.


In [6]:
# 6. 문서와 질문의 관련성을 검증하는 check_doc_relevance노드

 # - 수정 필요 : 기존에는 relevance를 확인 후 generate를 하거나, rewrite를 했다. 이제는 END를 return을 해야하는데, END는 노드가 아니라 Literal로 선언해도 작동을 안한다.
 #              이거는 Edge를 등록을 할 때 거기서 처리를 해줘야 한다. 

from langchain import hub
from typing import Literal
doc_relevance_prompt = hub.pull('langchain-ai/rag-document-relevance')

def check_doc_relevance(state: AgentState) -> Literal['generate', 'rewrite']:
    query = state['query']
    context = state['context']
    print(f'context == {context}')
    doc_relevance_chain = doc_relevance_prompt | llm
    response = doc_relevance_chain.invoke({'question': query, 'documents': context}) # 수정: context를 그대로 전달
    print(f'doc relevance response: {response}')
    
    # state를 설정하는 것이 아니라, 다음에 어디로 갈지 결정. 
    # 따라서 state를 return하면 안되고, score가  1이면 generate로 가고, 아니면 rewrite로 가도록 하겠다.
    if response['Score'] == 1:
        # return 'generate'
        return 'relevant'
    # return 'rewrite'
    return 'irrelevant'





In [7]:
# 7. rewrite 노드

from langchain_core.prompts import PromptTemplate
# output type을 str로만 나오게 한다 : 왜냐하면 state를 보면 query가 str이기 때문이다.
from langchain_core.output_parsers import StrOutputParser 


dictionary = ['사람과 관련된 표현 -> 거주자']

rewrite_prompt = PromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요
사전: {dictionary}
질문: {{query}}
""")

def rewrite(state: AgentState):
    query = state['query']
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()

    response = rewrite_chain.invoke({'query': query}) #response 가 BaseMessage 이므로 putput type을 str로 바꿔줘야 한다.
    return {'query': response}


In [8]:
# 8. 새로운 노드 추가 : 환각(hallucination) 검증 노드


# from langchain import hub
from langchain_core.prompts import PromptTemplate

hallucination_prompt = PromptTemplate.from_template("""
You are a teacher tasked with evaluating whether a student's answer is based on facts or not,
Given facts, which are excerpts from income tax law, and a student's answer;
If the student's answer is based on the facts, respond with "not hallucinated",
If the student's answer is not based on the facts, respond with "hallucinated".
                                                    
documents: {documents}
student_answer: {student_answer}
""")

hallucination_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

def check_hallucination(state: AgentState):
    answer = state['answer']
    context = state['context']
    context = [doc.page_content for doc in context]

    hallucination_chain = hallucination_prompt | hallucination_llm
    response = hallucination_chain.invoke({'student_answer': answer, 'documents': context})
    print(f'hallucination response: {response}')

    return response



In [9]:
query = '연봉이 5천만원인 거주자의 소득세'
context = retriever.invoke(query)
print('document 출력')
for document in context:
    print(document.page_content)
print('document 출력 끝')
generate_state = {'query': query, 'context': context}
answer = generate(generate_state)
print(f'answer == {answer}')
hallucination_state = {'answer': answer, 'context': context}

check_hallucination(hallucination_state)


document 출력
⑥ 제5항 단서에도 불구하고 장기주택저당차입금이 다음 각 호의 어느 하나에 해당하는 경우에는 연 800만원 대신 그 해당 각 호의 액면을 공개한도로 하여 제8항 본문을 적용한다.<신설 2014. 12. 23, 2013. 12. 31.>
  1. 차입금의 상환기간이 15년 이상 장기주택저당차입금의 이자를 대봉정림으로 정하고 교정림(이하 이 항에서 “고정금리라 한다”)이라고 하며 차입금을 대봉정림으로 정한 경우에 비거주자 분당상환 방식으로 상환하는 경우: 2천만원
  2. 차입금의 상환기간이 15년 이상 장기주택저당차입금의 이자를 고정금리로 지급하거나 그 차입금을 비거주자 분당상환으로 상환하는 경우: 1천800만원
  3. 차입금의 상환기간이 10년 이상 장기주택저당차입금의 이자를 고정금리로 지급하거나 그 차입금을 비거주자 분당상환으로 상환하는 경우: 600만원
  <삭제 2014. 1. 1.>
제1항 및 제5항에 따른 공지는 해당 거주자가 대봉정림으로 정한 바에 따라 신청한 경우에 적용하며, 공개액이 거주자의 해당 각 기간의 합산소득금액을 초과하는 경우 그 초과하는 금액은 없는 것으로 한다.<개정 2014. 1. 1., 2014. 1.>
<삭제 2014. 1. 1.>
<삭제 2014. 1. 1.>
제1항 및 제5항에 따른 공지는 “특별소득공제”란 한다.<개정 2014. 1. 1.>
특별소득공제에 관해서는 그 밖에 필요한 사항을 대봉정림으로 정한다.<개정 2014. 1. 1.>
제53조(생재를 같이 하는 부양가족의 범위와 판단기준) 제50조에 규정된 생계를 같이 하는 부양가족의 주민등록표 상의 동거가족으로서 해당 거주자의 주소 또는 거소에 실질적으로 생계를 같이 하는 사람으로 판단한다. 단, 자치법령에 의한 경우는 그러하지 아니하다.
  ① 거주자 또는 동거가족(친지부모; 입양자는 제외한다)에 취하, 질병의 양, 근무 또는 신자의 형 등의 본래의 주소 또는 거소에서 실질적으로 대봉정림으로 판단되는 사유에 해당할 경우에는 제1항의 세례를 같이 

AIMessage(content='hallucinated', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 2488, 'total_tokens': 2492, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9ba4454-5af0-4d7e-933f-165d8997b2be-0', usage_metadata={'input_tokens': 2488, 'output_tokens': 4, 'total_tokens': 2492, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})